In [9]:
from googleapiclient.discovery import build
import os
import json

# Load the secrets from the JSON file
with open('client_secrets.json', 'r') as file:
    secrets = json.load(file)

# Set up the API key and YouTube API client
api_key = secrets['api_key']  
youtube = build('youtube', 'v3', developerKey=api_key)

In [10]:
# Function to fetch video IDs
def get_video_ids(query, max_results=100):
    video_ids = []
    next_page_token = None

    while len(video_ids) < max_results:
        # Fetch search results
        request = youtube.search().list(
            part="id",
            q=query,
            type="video",
            maxResults=50,  # Adjust as needed (max 50 per request)
            pageToken=next_page_token
        )
        response = request.execute()

        # Extract video IDs
        for item in response.get('items', []):
            video_ids.append(item['id']['videoId'])
            if len(video_ids) >= max_results:
                break

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break

    return video_ids

In [11]:
# Example use
video_ids = get_video_ids("funny")  # Replace with your desired search query and optional max results (standard is 100)

['YnUU5U4lpSc', 'IOwLVfO_xZM', 'M-srjs_p2xE', 'e_9NhHz2V3w', 'bZdpihF44R0', 'VoU80UUXxHI', 'FRdgTgKxnx4', 'NNWMH-vRgOw', 'A8f8ajULAbw', 'cdmKVXynX3I', 'tk8qJiU1kbo', 'n0fd2MEC2rE', 'gcYrpkquExE', 'TH8w_c2jPG0', 'kBijGDSfUW8', 'D2DRAsKVPtM', '4siEToik4-w', 'iye3itnH9cs', 'jr9Xp1498oA', 'Ru7zVeY4EKs', 'P_1-Cpo1P0A', 'vzMyJ_lmMfg', 'UIhuGgET65o', 'HN6xEZOOGCg', 'uEvrbh_8UNU', 'BduIn6d2lhg', '8sYMjLAVcyA', '45N6-SbtV4E', 'TxHMDK-ca8s', 'FbXEbAtsufE', 'yJa__q2Z8Lg', 'VlZh-5UMFSI', 'rT1MTP-Rrt8', 'aEBxl6x5Ym8', 'mTKqSJrQxz8', 'ZvHagE3wDtI', 'Vu6T59lCheQ', '6LZ1NmPvNOU', 'xioYjDpP-sQ', '6TYFH8UXGm4', 'CuJRhKrq_IA', 'YAEBsmie_SA', 'v3mmI8K39pI', 'weNG3ZZ1LU8', 'EkhJPuPg38g', '7bnKr6XfcWc', 'KUjBpT92IP4', 'qnmb43iNq3w', 'WLIAzgw9IUg', 'pzZSbIqbvtA', 'Qr1lKlRWy18', '45N6-SbtV4E', 'gL4mYSHav-4', 'YI_B8SD_91k', '2rAlLeCJ_L0', 'YAEBsmie_SA', 'RI2t8k5V5AE', 'wVMSi1I8QbM', 'CyKUPqvbYM8', 'WjJcbHDY-bA', '_eRDDKoHP3Q', '0k1SPmGrFoU', 'eJ-ESo2CRS4', '7UwA0wQTlLY', '7bnKr6XfcWc', 'EscnTjs6jvE', 'ORd4HaRB